<a href="https://colab.research.google.com/github/Dkaattae/ML-Zoomcamp-2025/blob/main/08-deep-learning/MLZoomcamp_deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

--2025-11-30 21:39:08--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-30T22%3A33%3A28Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-30T21%3A32%3A31Z&ske=2025-11-30T22%3A33%3A28Z&sks=b&skv=2018-11-09&sig=ApAF7kBlsP%2BTEFwx0oeJw%2FbCpFF5nbDVPybyFGU%2FCiw%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NDU0MDU0OCwibmJmIjoxNzY0NTM4NzQ4LCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlv

In [1]:
import torch
from PIL import Image
import numpy as np

In [2]:
print(torch.__version__)

2.9.0+cu126


In [3]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
import os
from torch.utils.data import Dataset

In [5]:
import torchvision.models as models
from torchvision import transforms

In [6]:
class HairTypeDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = sorted(os.listdir(data_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_name in self.classes:
            label_dir = os.path.join(data_dir, label_name)
            for img_name in os.listdir(label_dir):
                self.image_paths.append(os.path.join(label_dir, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
input_size = 200

In [17]:
# ImageNet normalization values
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# Simple transforms - just resize and normalize
train_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Normalize(
        mean=mean,
        std=std
    ) # ImageNet normalization
])

val_transforms = transforms.Compose([
    transforms.Resize((input_size, input_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [18]:
from torch.utils.data import DataLoader

train_dataset = HairTypeDataset(
    data_dir='./data/train',
    transform=train_transforms
)

val_dataset = HairTypeDataset(
    data_dir='./data/test',
    transform=val_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)

In [10]:
import torch.nn as nn

class HairTypeClassifierMobileNet(nn.Module):
    def __init__(self, num_classes=2):
        super(HairTypeClassifierMobileNet, self).__init__()

        self.conv = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3),padding=1)
        self.relu1 = nn.ReLU()
        self.max_pooling = nn.MaxPool2d((2, 2))

        self.inner_layer = nn.Linear(32*100*100, 64)
        self.relu2 = nn.ReLU()
        self.output_layer = nn.Linear(64, 1)

    def forward(self, x):
        # x = self.base_model.features(x)
        x = self.conv(x)
        x = self.relu1(x)
        x = self.max_pooling(x)
        x = torch.flatten(x, 1)
        x = self.inner_layer(x)
        x = self.relu2(x)
        x = self.output_layer(x)
        return x


In [11]:
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = HairTypeClassifierMobileNet(num_classes=2)
model.to(device);

In [12]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

In [13]:
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 200, 200]             896
              ReLU-2         [-1, 32, 200, 200]               0
         MaxPool2d-3         [-1, 32, 100, 100]               0
            Linear-4                   [-1, 64]      20,480,064
              ReLU-5                   [-1, 64]               0
            Linear-6                    [-1, 1]              65
Total params: 20,481,025
Trainable params: 20,481,025
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 21.97
Params size (MB): 78.13
Estimated Total Size (MB): 100.56
----------------------------------------------------------------
Total parameters: 20481025


Total parameters: 20481025

In [19]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(val_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6699, Acc: 0.6504, Val Loss: 0.6142, Val Acc: 0.6468
Epoch 2/10, Loss: 0.5694, Acc: 0.6954, Val Loss: 0.8255, Val Acc: 0.6468
Epoch 3/10, Loss: 0.5615, Acc: 0.7203, Val Loss: 0.6200, Val Acc: 0.7015
Epoch 4/10, Loss: 0.5584, Acc: 0.7266, Val Loss: 0.5253, Val Acc: 0.7662
Epoch 5/10, Loss: 0.5488, Acc: 0.7129, Val Loss: 0.5191, Val Acc: 0.7463
Epoch 6/10, Loss: 0.5342, Acc: 0.7366, Val Loss: 0.5273, Val Acc: 0.7463
Epoch 7/10, Loss: 0.5011, Acc: 0.7341, Val Loss: 0.5471, Val Acc: 0.7512
Epoch 8/10, Loss: 0.4931, Acc: 0.7466, Val Loss: 0.5527, Val Acc: 0.7313
Epoch 9/10, Loss: 0.4930, Acc: 0.7615, Val Loss: 0.5062, Val Acc: 0.7861
Epoch 10/10, Loss: 0.4858, Acc: 0.7578, Val Loss: 0.6628, Val Acc: 0.6269


In [15]:
np.median(history['acc'])

np.float64(0.7865168539325842)

mean_acc = 0.7865168539325842

In [16]:
np.std(history['loss'])

np.float64(0.11164491853121432)

std_acc = 0.11164491853121432

add data augmentation, run transformer code snippet and data loader code snippet again. then the training for 10 epochs. not execute the model initiation code snippet.

In [20]:
np.mean(history['val_loss'])

np.float64(0.5900124226117607)

mean_val_loss = 0.5900124226117607

In [21]:
np.mean(history['val_acc'][6:])

np.float64(0.7238805970149255)

mean of last 5 epochs test accuracy is: 0.7238805970149255